In [44]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import os
import torch
import numpy as np
import torch.nn as nn
import pandas as pd
import matplotlib.pyplot as plt
from torch.utils.data import Dataset,DataLoader
#from datasets import load_dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from torch.utils.tensorboard import SummaryWriter

In [46]:
#import dataset
ds=pd.read_csv('Reviews.csv')

In [47]:
#pure_ds=pd.read_csv('/kaggle/input/amazon-product-reviews/Reviews.csv')

In [48]:
ds.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [49]:
ds.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [50]:
ds=ds.dropna(how='any')

In [51]:
ds.drop(['ProfileName','HelpfulnessNumerator','HelpfulnessDenominator','Summary','Text'],axis=1,inplace=True)

# encoder

In [52]:
object_encoders={}
columns=['ProductId','UserId']
for col in columns:
    le=LabelEncoder()
    ds[col]=le.fit_transform(ds[col])
    #storing the feature values and its encoded values
    object_encoders[col]=le

In [53]:
#change the time stamp into datatime
ds['Time']=pd.to_datetime(ds['Time'],unit='s',utc=True)
ds['Time']=ds['Time'].dt.tz_convert('America/New_York')

In [54]:
#year
ds['year']=ds['Time'].dt.year
#month
ds['month']=ds['Time'].dt.month
#week
ds['day_of_week']=ds['Time'].dt.dayofweek
#hour
ds['hour']=ds['Time'].dt.time

In [55]:
ds.isnull().sum()

Id             0
ProductId      0
UserId         0
Score          0
Time           0
year           0
month          0
day_of_week    0
hour           0
dtype: int64

In [56]:
clean_data=ds.drop(['Time'],axis=1)

In [57]:
clean_data.drop(['Id'],axis=1,inplace=True)

In [58]:
clean_data['hour']=clean_data['hour'].apply(lambda x:x.hour )

In [59]:
clean_data['Recomended']=clean_data['Score']>=3

In [60]:
clean_data.drop(['Score'],axis=1,inplace=True)

In [61]:
def mapping_function_year(year):
    return year-1999

In [62]:
clean_data['year']=clean_data['year'].apply(mapping_function_year)

In [63]:
def mapping_function_hour(hour):
    return hour-19

In [64]:
clean_data['hour'].value_counts()

hour
20    382504
19    185897
Name: count, dtype: int64

In [65]:
clean_data['hour']=clean_data['hour'].apply(mapping_function_hour)

In [66]:
clean_data['Recomended']=clean_data['Recomended'].astype(int)

In [67]:
clean_data

,ProductId,UserId,year,month,day_of_week,hour,Recomended
0,27619,188633,12,4,1,1,1
1,72383,25104,13,9,3,1,0
2,15267,210468,9,8,6,1,1
3,19718,152622,12,6,6,1,0
4,69007,57800,13,10,5,1,1
...,...,...,...,...,...,...,...
568449,29548,83886,12,3,1,0,1
568450,49388,169555,13,3,3,0,0
568451,57631,4347,13,2,0,0,1
568452,57631,169715,13,3,0,1,1


In [68]:
#split into train and test
train,test=train_test_split(
    clean_data,
    random_state=42,
    test_size=0.2,
    stratify=clean_data['Recomended']
)

In [69]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [96]:
class Mydataset(Dataset):
    def __init__(self,data):
        super().__init__()
        self.data=data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self,idx):
        #element to extract
        item=self.data.iloc[idx]
        
        return {
            "product_id":torch.tensor([item['ProductId']],dtype=torch.long).to(device),
            "user_id":torch.tensor([item['UserId']],dtype=torch.long).to(device) ,
            "year":torch.tensor([item['year']],dtype=torch.long).to(device) ,
            "month":torch.tensor([item['month']],dtype=torch.long).to(device) ,
            "day_of_week":torch.tensor([item['day_of_week']],dtype=torch.long).to(device) ,
            "hour":torch.tensor([item['hour']],dtype=torch.long).to(device) ,
            "recomended":torch.tensor([item['Recomended']],dtype=torch.float).to(device)
        }

In [97]:
train_dataset=Mydataset(train)
test_dataset=Mydataset(test)

In [98]:
#making the train dataloader
train_dataloader=DataLoader(
    dataset=train_dataset,
    batch_size=32
)

#making test dataloader
test_dataloader=DataLoader(
    dataset=test_dataset,
    batch_size=32
)

In [99]:
for batch in train_dataloader:
    print(batch['product_id'].shape)
    print(batch['user_id'].shape)
    break

torch.Size([32, 1])
torch.Size([32, 1])


In [100]:
from main import WideDeep,WD_Config

In [101]:
config=WD_Config()
config.num_product=clean_data['ProductId'].nunique()
config.num_users=clean_data['UserId'].nunique() 
config.num_year=clean_data["year"].nunique()
config.num_time_day=clean_data["hour"].nunique()
config.num_month=clean_data['month'].nunique()+1   #since in this column it start from 1
config.num_day_week=clean_data["day_of_week"].nunique()
config.embedding_dim=100

In [102]:
model=WideDeep(config)

In [103]:
optim=torch.optim.AdamW(model.parameters(),lr=0.01,weight_decay=0.01)  #the deep component
loss_fn=torch.nn.BCEWithLogitsLoss() 

In [104]:
# extraxt one batch 
for batch in train_dataloader:
    one_batch=batch
    break

In [105]:
tot_epoch=1

In [110]:
for epoch in range(tot_epoch):
    product_id=one_batch['product_id']
    user_id=one_batch['user_id']
    year=one_batch['year']
    month=one_batch['month']
    day_of_week=one_batch['day_of_week']
    hour=one_batch['hour']
    recomended=one_batch['recomended']
    
    logits=model(
         product_id,
        user_id,
        year,
        month,
        day_of_week,
        hour
    )
    
    loss=loss_fn(logits,recomended)
    print(f"loss is: {loss.item()}")
    print(f"the wide out put is:{model.wide_output.shape} ")
    optim.zero_grad()
    loss.backward()
    
    optim.step()

product id type is: torch.int64
user id type is: torch.int64
product id embed: torch.Size([32, 1, 100])
user input embed: torch.Size([32, 1, 100])
year embed: torch.Size([32, 1, 100])
time of month: torch.Size([32, 1, 100])
day of week shape:torch.Size([32, 1, 100])
time of day embed: torch.Size([32, 1, 100])
loss is: 0.5526117086410522
the wide out put is:torch.Size([32, 1]) 
